# **CREATING SQL QUERIES ON THE OLYMPICS DATASET FROM KAGGLE**

# 1\. How many olympics games have been held?

In [40]:
-- Getting the distinct games played and storing in a temporal table
with t1 as (
    select games, count(distinct games) as total_count from olympics_events group by games
)

-- Summing up the total number of games
select sum(total_count) as Total_Olympic_games from t1;

(1 row affected)

Total execution time: 00:00:00.262

Total_Olympic_games
51


# 2\. List down all Olympics games held so far and their various cities they were held in .

In [41]:
-- Getting the distinct games and city 
select distinct games, city from olympics_events order by games ;

(52 rows affected)

Total execution time: 00:00:00.113

games,city
1896 Summer,Athina
1900 Summer,Paris
1904 Summer,St. Louis
1906 Summer,Athina
1908 Summer,London
1912 Summer,Stockholm
1920 Summer,Antwerpen
1924 Summer,Paris
1924 Winter,Chamonix
1928 Summer,Amsterdam


# 3\. Give the total number of nations that participated in each olympic games

In [42]:
select distinct games , count(distinct noc) as total_nations from olympics_events group by games order by games;

(51 rows affected)

Total execution time: 00:00:00.082

games,total_nations
1896 Summer,12
1900 Summer,31
1904 Summer,15
1906 Summer,21
1908 Summer,22
1912 Summer,29
1920 Summer,29
1924 Summer,45
1924 Winter,19
1928 Summer,46


# 4\. Which olympic games had the highest and lowest number of participated nations respectively

In [43]:
with t1 as (select distinct games , count(distinct noc) as total_nations from olympics_events group by games),

t2 as (select concat(games, '-' , total_nations) as Highest_country from t1 
where total_nations = (select max(total_nations) from t1)),

t3 as (select  concat(games, '-' , total_nations) as Lowest_country from t1
where total_nations  = (select min(total_nations) from t1))

select * from t2 join t3 on t2.Highest_country <> t3.Lowest_country;


(1 row affected)

Total execution time: 00:00:00.280

Highest_country,Lowest_country
2016 Summer-207,1896 Summer-12


# 5\. List the number of countries that have participated in all every olympic games being held.

In [44]:
-- Joining the two tables to get the full country names. 
with t1 as (
    select distinct o.games, n.region as country from olympics_events as o
    join noc_regions as n on o.noc = n.noc       
   ),
-- Getting the total number of distinct olympic games held 
t2 as (select count(distinct games) as total_games from t1),

t3 as (select games, country from t1 group by games, country ),

-- Getting the total participation for each country
t4 as (select country , count(country) as Total_Participations from t3 group by country )

-- Comparing the participation total for each country with the total games, to get the country
-- that particpated in all olympic games 
select country, total_participations from t4 join t2 on t4.Total_Participations = t2.total_games
order by country;


(4 rows affected)

Total execution time: 00:00:00.415

country,total_participations
France,51
Italy,51
Switzerland,51
UK,51


# 6\. Find the sports that have ocurred in all the summer olympic games being held

In [45]:
-- Filtering the dataset to get just olympic events that occured in the summer 
with t as (select * from olympics_events where season = 'summer'), 
-- Count without duplicates to know how many summer olympic games have been held in total
t1 as (select count(distinct games) as total_summer_games from t), 
-- Removing duplicates to get just the distinct sports and games
t2 as (select distinct games, sport from t), 

t3 as (select sport, count(sport) as total_occurence from t2 group by sport )
-- comparing the total count of each sport with the total number of summer olympic games held 
select sport, total_occurence from t3 join t1 on t3.total_occurence = t1.total_summer_games


(5 rows affected)

Total execution time: 00:00:00.131

sport,total_occurence
Fencing,29
Athletics,29
Cycling,29
Swimming,29
Gymnastics,29


# 7\. Which sports where the second most occured sports in the summer olympic games

In [46]:
-- Filtering the dataset to get just olympic events that occured in the summer 
with t as (select * from olympics_events where season = 'summer'), 
-- Removing duplicates to get just the distinct sports and games
t1 as (select distinct games, sport from t), 

t2 as (select sport, count(sport) as total_occurence from t1 group by sport),

-- Ranking the total occurrence 
t3 as (select sport, total_occurence, dense_rank() over (order by total_occurence desc) as Ranks from t2)

select sport, total_occurence from t3 where ranks = 2;



(2 rows affected)

Total execution time: 00:00:00.074

sport,total_occurence
Wrestling,28
Rowing,28


# 8\. Find the total number of sports played in each olympic games

In [47]:
-- Filter just the distinct games and sports 
with t as (
    select distinct games, sport from olympics_events
) 
-- counting the sports by games and ordering them from highest to lowest 
select games , count(sport) as total_sport_played 
from t group by games 
order by total_sport_played desc

(51 rows affected)

Total execution time: 00:00:00.066

games,total_sport_played
2000 Summer,34
2016 Summer,34
2004 Summer,34
2008 Summer,34
2012 Summer,32
1996 Summer,31
1992 Summer,29
1988 Summer,27
1984 Summer,25
1920 Summer,25


# 9\. Find the oldest athletes to win a golden medal

In [48]:
-- Filtering to get data just for gold medal and ages which are not null
with t as (
    select games, name, age, sex from olympics_events where medal = 'Gold' and age <> 'NA'
), 

t1 as (select *, dense_rank() over(order by age desc) as age_rank from t)

-- Getting the oldest atlethes;
select games, name as Oldest_athlethes, age, sex from t1 
where age_rank = (select min(age_rank) from t1) 

(2 rows affected)

Total execution time: 00:00:00.719

games,Oldest_athlethes,age,sex
1912 Summer,Oscar Gomer Swahn,64,M
1904 Summer,Charles Jacobus,64,M


# 10\. Find the youngest athletes to win a golden medal

In [49]:

-- Filtering to get data just for gold medal and ages which are not nulls
with t as (
    select games, name, age, sex from olympics_events where medal = 'Gold' and age <> 'NA'
), 

t1 as (select *, dense_rank() over(order by age desc) as age_rank from t)

-- Getting the Youngest atlethes;
  select games, name as Youngest_athlethes, age, sex from t1 
  where age_rank = (select max(age_rank) from t1) ;


(7 rows affected)

Total execution time: 00:00:01.205

games,Youngest_athlethes,age,sex
1920 Summer,Aileen Muriel Riggin (-Soule),13,F
1992 Summer,Fu Mingxia,13,F
1994 Winter,Kim Yun-Mi,13,F
1960 Summer,Donna Elizabeth de Varona (-Pinto),13,F
1936 Summer,Marjorie Gestring (-Redlick),13,F
1928 Summer,Hans Bourquin,13,M
1960 Summer,Klaus Zerta,13,M


# 11\. Find the ratio of female and male atheletes who participated in all the olympic games

In [50]:
-- Using the with clause to create a temporal tables
with cte as (
    select games,  name,  sex from olympics_events 
), 
-- Count only females per olympic games
t1 as (select games, count(sex) as female_count_per_game from cte where sex = 'F' group by games),
-- Count only males per olympic games
t2 as (select games, count(sex) as male_count_per_game from cte where sex = 'M' group by games),
-- Getting the total females for all the games
t3 as (select sum(t1.female_count_per_game) as total_females from t1),
-- Getting the total males for all the games
t4 as (select sum(t2.male_count_per_game) as total_males from t2),

-- Joining the two tables to get the two total values of males and females
t5 as (select t3.*, t4.* from t3 join t4 on t4.total_males <> t3.total_females)

-- Changing the datatype to float since the sql server does automatic rounding 
select Concat(total_females/total_females, ' : ' , round(cast(total_males as float)/cast(total_females as float), 2)) 
as Ratio from t5;

(1 row affected)

Total execution time: 00:00:00.095

Ratio
1 : 2.64


# 12.  Find the top 5 athletes who has won the most golden medal

In [51]:
-- Filtering to get just the information about gold medals 
with cte as ( select  * from olympics_events where Medal = 'Gold' ),

-- Counting for each person by the names
t1 as (select name, count(medal) as total_medal from cte group by name ), 
-- Ranking the total from hightest to lowest 
t2 as (select *, dense_rank() over (order by total_medal desc) as rnk from t1)
-- Getting just the top 5 highest 
select name, total_medal from t2 where rnk <= 5;

(18 rows affected)

Total execution time: 00:00:00.120

name,total_medal
"Michael Fred Phelps, II",23
"Raymond Clarence ""Ray"" Ewry",10
Paavo Johannes Nurmi,9
Larysa Semenivna Latynina (Diriy-),9
Mark Andrew Spitz,9
"Frederick Carlton ""Carl"" Lewis",9
"Jennifer Elisabeth ""Jenny"" Thompson (-Cumpelik)",8
Birgit Fischer-Schmidt,8
"Matthew Nicholas ""Matt"" Biondi",8
Ole Einar Bjrndalen,8


# 13.  Find the top 5 athletes with the most medals (Gold, Bronze, Silver)

In [52]:
-- Filtering to get information where there is data on medal
with cte as ( select  * from olympics_events where Medal <> 'NA' ), 
-- Counting by name
t1 as (select name , count(medal) as total_medal from cte group by name),
-- Ranking by the total in descending order
t2 as (select *, dense_rank() over (order by total_medal desc) as rnk from t1)
-- Fetching just the top 5. 
select name, total_medal from t2 where rnk <= 5;


(15 rows affected)

Total execution time: 00:00:00.270

name,total_medal
"Michael Fred Phelps, II",28
Larysa Semenivna Latynina (Diriy-),18
Nikolay Yefimovich Andrianov,15
Ole Einar Bjrndalen,13
Borys Anfiyanovych Shakhlin,13
Edoardo Mangiarotti,13
Takashi Ono,13
Ryan Steven Lochte,12
Sawao Kato,12
Birgit Fischer-Schmidt,12


# 14\. Find the top five (5) countries with the most medals.(Gold/Bronze/Silver)

In [53]:
-- Filtering to get only where the is information about the medal
-- Joining with the other table to get the full country name 
with cte as (select o.name, n.region, o.games, o.medal from olympics_events o
                    join noc_regions n on n.noc = o.noc 
                    where medal <> 'NA'
            ),

-- Counting the total medal by country 
t2 as (select region, count(medal) as total_count from cte group by region ), 
-- Ranking the counts from highest to lowest 
t3 as (select *, dense_rank() over (order by total_count desc) as rnk from t2)
-- Fetching the top 5 countries. 
select region, total_count from t3 where rnk <= 5;

(5 rows affected)

Total execution time: 00:00:00.086

region,total_count
USA,5637
Russia,3947
Germany,3756
UK,2068
France,1777


# 15.  List out the number of  Bronze, Gold and Silver medals for each country , with the Bronze, Gold, Silver in column level

In [54]:
with cte as (
    -- filtering just where there is data for medal and using as base data
        select * from 
            (
            select n.region, o.medal from olympics_events o
                                    join noc_regions n on n.noc = o.noc 
                                    where medal <> 'NA'

        ) as output_data 

        PIVOT

        (
            -- counting the various medal and creating the columns
            count(medal) 
            for medal in ([Gold], [Bronze], [Silver]) 
            
        ) as final_output 

)

select * from cte order by Gold desc;

(136 rows affected)

Total execution time: 00:00:00.107

region,Gold,Bronze,Silver
USA,2638,1358,1641
Russia,1599,1178,1170
Germany,1301,1260,1195
UK,678,651,739
Italy,575,531,531
France,501,666,610
Sweden,479,535,522
Canada,463,451,438
Hungary,432,371,332
Norway,378,294,361


# 16.   Give total gold, silver and bronze medals won by each country with respect to each olympic games.

In [57]:
   with cte as (
  -- filtering just where there is data for medal and using as base data
  -- Joining the two tables to get the full names of the countries.
     select * from 
            (
            select o.games, n.region, o.medal from olympics_events o
                                    join noc_regions n on n.noc = o.noc 
                                    where medal <> 'NA'

        ) as output_data 

        PIVOT

        (
            -- counting the various medal and creating the columns
            count(medal) 
            for medal in ([Gold], [Bronze], [Silver]) 
            
        ) as final_output 
     )

     select * from cte order by Gold desc;

(1640 rows affected)

Total execution time: 00:00:00.339

games,region,Gold,Bronze,Silver
1980 Summer,Russia,187,126,129
1984 Summer,USA,186,50,116
1996 Summer,USA,159,52,48
1908 Summer,UK,147,90,131
2012 Summer,USA,145,46,57
2016 Summer,USA,139,71,54
1988 Summer,Russia,134,99,67
2000 Summer,USA,130,51,61
1904 Summer,USA,128,125,141
2008 Summer,USA,127,80,110


# 17.  List out each country with their total gold, bronze and silver medals in each olympic game and also the total of all                 the  medals they won for each olympic games

In [75]:
      with cte as (
  -- filtering just where there is data for medal and using as base data
     select * from 
            (
            select o.games, n.region, o.medal from olympics_events o
                                    join noc_regions n on n.noc = o.noc 
                                    where medal <> 'NA'

        ) as output_data 

        PIVOT

        (
            -- counting the various medal and creating the columns
            count(medal) 
            for medal in ([Gold], [Bronze], [Silver]) 
            
        ) as final_output 
     ),  
-- Using partition by, to get just the sum of the gold, silver, bronze for each country in each olympic games 
 t1 as  ( select games, region, Gold, Bronze, Silver, sum(gold + bronze + silver) over (partition by games, region, gold, bronze, silver)
        as total from cte 
    )

select * from t1 order by total desc;



(1640 rows affected)

Total execution time: 00:00:00.514

games,region,Gold,Bronze,Silver,total
1980 Summer,Russia,187,126,129,442
1904 Summer,USA,128,125,141,394
1908 Summer,UK,147,90,131,368
1984 Summer,USA,186,50,116,352
2008 Summer,USA,127,80,110,317
1988 Summer,Russia,134,99,67,300
1988 Summer,Germany,111,94,91,296
1976 Summer,Russia,114,77,95,286
1976 Summer,Germany,123,74,76,273
1980 Summer,Germany,115,61,88,264


# 18\. Identify the top 10 best countries with most medals in each olympic games including their total number of Gold,                Bronze and Silver medals for each olympic game

In [68]:
   with cte as (
  -- filtering just where there is data for medal and using as base data
     select * from 
            (
            select o.games, n.region, o.medal from olympics_events o
                                    join noc_regions n on n.noc = o.noc 
                                    where medal <> 'NA'

        ) as output_data 

        PIVOT

        (
            -- counting the various medal and creating the columns
            count(medal) 
            for medal in ([Gold], [Bronze], [Silver]) 
            
        ) as final_output 
     ),  
-- Using partition by, to get just the sum of the gold, silver, bronze for each country in each olympic games 
 t1 as  ( select *, sum(gold + bronze + silver) over (partition by games, region, gold, bronze, silver)
        as total from cte 
    ), 

t2 as (select *, dense_rank() over (order by total desc) as rnk from t1)

select games, region, Gold, Bronze, Silver, total from t2 where rnk <= 10;



(11 rows affected)

Total execution time: 00:00:00.345

games,region,Gold,Bronze,Silver,total
1980 Summer,Russia,187,126,129,442
1904 Summer,USA,128,125,141,394
1908 Summer,UK,147,90,131,368
1984 Summer,USA,186,50,116,352
2008 Summer,USA,127,80,110,317
1988 Summer,Russia,134,99,67,300
1988 Summer,Germany,111,94,91,296
1976 Summer,Russia,114,77,95,286
1976 Summer,Germany,123,74,76,273
1980 Summer,Germany,115,61,88,264


# 19\. Which season has more medals won

In [84]:
with cte as (select season, count(medal) as total_medal_season from  olympics_events where medal <> 'NA'
group by season ),

t1 as (select * , dense_rank() over(order by total_medal_season desc) as rnk from cte)

select season , total_medal_season from t1 where rnk = (select min(rnk) from t1);

(1 row affected)

Total execution time: 00:00:00.116

season,total_medal_season
Summer,34088


# 20\. Give the total number of Golds, Silver and Bronze won in each season

In [76]:

with cte as (
    select * from 
        (
            select season, medal from olympics_events where medal <> 'NA' 
            
        ) as base_data 
    PIVOT
        (
            count(medal)
            for medal in ([Gold], [Silver], [Bronze])
        )  as output
)

select *, sum(gold + silver + bronze) over(partition by season) as total from cte;


(2 rows affected)

Total execution time: 00:00:00.159

season,Gold,Silver,Bronze,total
Summer,11459,11220,11409,34088
Winter,1913,1896,1886,5695
